In [3]:
import websocket
import json
import time

# Diccionario para rastrear el último precio de cada activo
ultimo_precio = {}

def on_message(ws, message):
    try:
        msg = json.loads(message)
        symbol = msg['s'].replace('USDT', '').lower()
        
        # Importamos los buffers globales del main
        from main_godzilla import flujo_deltas, volumen_total
        
        if symbol in flujo_deltas:
            p, q = float(msg['p']), float(msg['q'])
            ultimo_precio[symbol] = p  # Guardamos el precio actual
            
            valor_trade = p * q
            # Registro de Delta (Dirección)
            delta = valor_trade if not msg['m'] else -valor_trade
            flujo_deltas[symbol].append(delta)
            
            # Registro de Volumen Total (Fuerza)
            volumen_total[symbol].append(valor_trade)
    except: pass

def iniciar_socket_con_reconexion(watchlist):
    while True:
        try:
            streams = "/".join([f"{t.lower()}usdt@trade" for t in watchlist])
            url = f"wss://stream.binance.com:9443/ws/{streams}"
            ws = websocket.WebSocketApp(url, on_message=on_message)
            ws.run_forever()
        except: time.sleep(5)
        